In [3]:
library(CoinCalc)

In [4]:
setwd("...")


In [5]:
CC.eca.ts

function (seriesA, seriesB, alpha = 0.05, delT = 0, sym = FALSE, 
    tau = 0, sigtest = "poisson", reps = 1000) 
{
    if (length(table(seriesA)) > 2) {
        print("|-------------    ERROR #1    -------------|")
        print("|       Time seriesA is not binary         |")
        print("|      (or the number of events=0)!        |")
        print("| Use CC.binarize() to preprocess seriesA  |")
        print("|------------------------------------------|")
        return()
    }
    if (length(table(seriesB)) > 2) {
        print("|-------------    ERROR #1    -------------|")
        print("|       Time seriesB is not binary         |")
        print("|      (or the number of events=0)!        |")
        print("| Use CC.binarize() to preprocess seriesA  |")
        print("|------------------------------------------|")
        return()
    }
    if (tau < 0 || delT < 0) {
        print("|-------------    ERROR #2    -------------|")
        print("|    The offset (tau) or delta T (delT)    |")
        print("|              is negative.                |")
        print("|------------------------------------------|")
        return()
    }
    if (length(seriesA) != length(seriesB)) {
        print("|-------------    ERROR #4    -------------|")
        print("|    lengths of series A and B differ      |")
        print("|------------------------------------------|")
        return()
    }
    if (!is.vector(seriesA) && !is.matrix(seriesA)) {
        print("|-------------    ERROR #5    -------------|")
        print("| series A is neither vector nor matrix    |")
        print("|------------------------------------------|")
        return()
    }
    if (!is.vector(seriesB) && !is.matrix(seriesB)) {
        print("|-------------    ERROR #5    -------------|")
        print("| series B is neither vector nor matrix    |")
        print("|------------------------------------------|")
        return()
    }
    if (any(is.na(seriesA)) && sigtest == "surrogate") {
        print("|-------------    ERROR #6    -------------|")
        print("|          seriesA contains NAs.           |")
        print("| surrogate test not allowed, use poisson  |")
        print("|------------------------------------------|")
        return()
    }
    if (any(is.na(seriesB)) && sigtest == "surrogate") {
        print("|-------------    ERROR #6    -------------|")
        print("|          seriesB contains NAs.           |")
        print("| surrogate test not allowed, use poisson  |")
        print("|------------------------------------------|")
        return()
    }
    if (any(is.na(seriesB)) && sigtest == "shuffle") {
        print("|-------------    ERROR #6    -------------|")
        print("|    seriesB or seriesA contains NAs.      |")
        print("| shuffle test not allowed, use poisson    |")
        print("|------------------------------------------|")
        return()
    }
    if (sum(is.na(seriesA)) == length(seriesA) || sum(is.na(seriesB)) == 
        length(seriesB)) {
        CA_out = list(NA, NA, NA, NA, NA, NA)
        names(CA_out) = c("NH precursor", "NH trigger", "p-value precursor", 
            "p-value trigger", "precursor coincidence rate", 
            "trigger coincidence rate")
        return(CA_out)
    }
    seriesA[is.na(seriesB)] = NA
    seriesB[is.na(seriesA)] = NA
    bindata = matrix(NA, 2, length(seriesA))
    bindata[1, ] = seriesA
    bindata[2, ] = seriesB
    rownames(bindata) = c("seriesA", "seriesB")
    Tlen = length(bindata[1, !is.na(bindata[1, ])])
    steplen = length(seriesA)
    N_A = as.numeric(Tlen - table(bindata[1, ] == 1)[1])
    N_B = as.numeric(Tlen - table(bindata[2, ] == 1)[1])
    K_prec = 0
    for (step in 1:steplen) {
        if (is.na(bindata[1, step])) {
            next
        }
        if (bindata[1, step] == 1) {
            start = step - tau - (delT)
            if (sym == TRUE) {
                end = step - tau + (delT)
            }
            if (sym == FALSE) {
     

In [6]:
authors_ecs_extracted <- function(seriesA, seriesB, delT = 0, tau = 0){
    bindata = matrix(NA, 2, length(seriesA))
    bindata[1, ] = seriesA
    bindata[2, ] = seriesB
    rownames(bindata) = c("seriesA", "seriesB")
    Tlen = length(bindata[1, !is.na(bindata[1, ])])
    steplen = length(seriesA)
    N_A = as.numeric(Tlen - table(bindata[1, ] == 1)[1])
    N_B = as.numeric(Tlen - table(bindata[2, ] == 1)[1])
    K_prec = 0
    for (step in 1:steplen) {
        if (bindata[1, step] == 1) {
            start = step - tau - (delT)
            end = step - tau
            if (start < 1 & end >= 1) {
                start = 1
            }
            if (start < 1 & end < 1) {
                next
            }
            if (start > steplen) {
                next
            }
            if (end > steplen) {
                end = steplen
            }
            if (is.element(1, bindata[2, start:end])) {
                K_prec = K_prec + 1
            }
        }
    }
    CRprec = K_prec/N_A
    K_trigg = 0
    for (step in 1:steplen) {
        if (bindata[2, step] == 1) {
            
            start = step + tau
            end = step + tau + (delT)
            if (start < 1 & end >= 1) {
                start = 1
            }
            if (start < 1 & end < 1) {
                next
            }
            if (start > steplen) {
                next
            }
            if (end > steplen) {
                end = steplen
            }
            if (is.element(1, bindata[1, start:end])) {
                K_trigg = K_trigg + 1
            }
        }
    }
    CRtrigg = K_trigg/N_B
    res = list(CRprec, CRtrigg)
    names(res) = c('CRprec', 'CRtrigg')
    return(res)
}

In [7]:
monsoonEventDf <- read.csv("monsoonEventDf.csv", header=TRUE)
monsoonEventDf <- monsoonEventDf[, colnames(monsoonEventDf) != 'DateTime'] 
monsoonEventDf <- sapply(monsoonEventDf[, 1:dim(monsoonEventDf)[2]], function(x) as.character(x) == 'True')

In [8]:
find_eca_for_all_combinations <- function(es_mat){
    number_of_grid_points <- 20
    # number_of_grid_points <-3
    r_precursor_i_j <- matrix(nrow=number_of_grid_points, ncol=number_of_grid_points)
    r_trigger_i_j <- matrix(nrow=number_of_grid_points, ncol=number_of_grid_points)
    for(i in 1:number_of_grid_points){
        for(j in 1:number_of_grid_points){
            # print(paste(i,j,sep=' '))
            eca_res <- authors_ecs_extracted(es_mat[, j], es_mat[, i], delT =60)
            r_precursor_i_j[i,j] <- eca_res$`CRprec`
            r_trigger_i_j[i,j] <- eca_res$`CRtrigg`
        }
    }
    EC_p_max <- as.data.frame(pmax(r_precursor_i_j, t(r_precursor_i_j)), row.names=colnames(es_mat), col.names=colnames(es_mat))
    EC_p_mean <- as.data.frame((r_precursor_i_j + t(r_precursor_i_j)) /2, row.names=colnames(es_mat), col.names=colnames(es_mat))
    EC_t_max <- as.data.frame(pmax(r_trigger_i_j, t(r_trigger_i_j)), row.names=colnames(es_mat), col.names=colnames(es_mat))
    EC_t_mean <- as.data.frame((r_trigger_i_j + t(r_trigger_i_j))/2, row.names=colnames(es_mat), col.names=colnames(es_mat))
    res <- list(EC_p_max, EC_p_mean, EC_t_max, EC_t_mean)
    names(res) <- c("EC_p_max", "EC_p_mean", "EC_t_max", "EC_t_mean")
    return(res)
}


In [9]:
start.time <- Sys.time()
res <- find_eca_for_all_combinations(monsoonEventDf)
end.time <- Sys.time()
EC_p_max<- res[1]
EC_p_mean<- res[2]
EC_t_max<- res[3]
EC_t_mean<- res[4]
time.taken <- round(end.time - start.time,2)
time.taken


Time difference of 1.02 mins

In [10]:
EC_p_max

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
1.0000000,0.9565217,0.9117647,0.9555035,0.7256637,0.8611111,0.9473684,0.9344262,0.9037123,0.8294118,0.8722222,0.8527607,0.9402810,0.9229739,0.8163265,0.8205128,0.8269231,0.9121780,0.9160924,0.8134328
0.9565217,1.0000000,0.7352941,0.9627329,0.3362832,0.4537037,0.2631579,0.9254658,0.8447205,0.4352941,0.3664596,0.3251534,0.9689441,0.7639752,0.3061224,0.3913043,0.3167702,0.9192547,0.8633540,0.3507463
0.9117647,0.7352941,1.0000000,1.0000000,0.9411765,0.8529412,0.5087719,0.9411765,1.0000000,0.9117647,0.7647059,0.3235294,0.9705882,0.7647059,0.4705882,0.6764706,0.3823529,0.9117647,0.9117647,0.3823529
0.9555035,0.9627329,1.0000000,1.0000000,0.8938053,0.9814815,0.9824561,0.9597403,0.9750580,0.9588235,0.9444444,0.9447853,0.9592461,0.9785666,0.9183673,0.9358974,0.9759615,0.9542857,0.9708148,0.9104478
0.7256637,0.3362832,0.9411765,0.8938053,1.0000000,0.5833333,0.7192982,0.8407080,0.8938053,0.4867257,0.4955752,0.3805310,0.9026549,0.6725664,0.4336283,0.4070796,0.3008850,0.9026549,0.8495575,0.3628319
0.8611111,0.4537037,0.8529412,0.9814815,0.5833333,1.0000000,0.6666667,0.9629630,0.9629630,0.5462963,0.6666667,0.4166667,0.9629630,0.6944444,0.5370370,0.6851852,0.4722222,0.9537037,0.9351852,0.5185185
0.9473684,0.2631579,0.5087719,0.9824561,0.7192982,0.6666667,1.0000000,0.9473684,0.9473684,0.7017544,0.7894737,0.7368421,0.9473684,0.5438596,0.6315789,0.7543860,0.6315789,0.9473684,0.9122807,0.7368421
0.9344262,0.9254658,0.9411765,0.9597403,0.8407080,0.9629630,0.9473684,1.0000000,0.9889791,0.9529412,0.9833333,1.0000000,0.9805195,0.9899531,0.9387755,0.9529915,0.9759615,0.9781818,0.9902716,0.9029851
0.9037123,0.8447205,1.0000000,0.9750580,0.8938053,0.9629630,0.9473684,0.9889791,1.0000000,0.9647059,0.9555556,0.9447853,0.9901392,0.9718687,0.9183673,0.9401709,0.9423077,0.9814385,0.9635184,0.8880597
0.8294118,0.4352941,0.9117647,0.9588235,0.4867257,0.5462963,0.7017544,0.9529412,0.9647059,1.0000000,0.5352941,0.3190184,0.9764706,0.7058824,0.3673469,0.5529412,0.4176471,0.9823529,0.9470588,0.4104478
